**Etudiants :** <br/>
3701123 - Pascal Zhuang<br/>
3416447 - Mohand BENCHABANE

# Recommenders 3 -- Sequence Recommenders (45m) 

## Goals of this practical:

- Understand the sequence recommendation framework (~5min)
- Load/Format dataset (~5min)
- Understand/train the prod2vec model (~10min)
- Evaluate (~10min)
- Visualize (~10min)
- Fiddle (~5min)



In [10]:
#! pip install gensim --upgrade

In [11]:
import pandas as pd
import numpy as np

# Sequence Recommenders:

> What will you click next ?

The sequence recommendation setting is a particular case of the implicit collaborative filtering setting. Given a sequence of items $i_0,i_1,...,i_n$ the goal is to predict the $i_{(n+1)},...$ items the user will consume. Playlist continuation is a neat use case of sequence recommenders. You've been listening to those songs, what can you listen to now ?


This setting differs from the classical collaborative filtering because the history is the recent trace and not the full saved interactions. Also, it's possible to do sequence recommendation without any specific latent user profile. 

#### Here we propose to explore this unpersonalized sequence recommandation

## Data used : [smallest movie-lens dataset](https://grouplens.org/datasets/movielens/)

Here we'll use the same data as before but instead of seeing $(user,item,rating)$ triplets or a $(user,item)$ interaction , we'll see item sequences: $user: [item, item,...]$

## Loading Data (same as before but in chronological order):

In [12]:
ratings = pd.read_csv("dataset/ratings.csv")
ratings = ratings.sort_values("timestamp",ascending=True)
print(ratings.iloc[0]["timestamp"] < ratings.iloc[-1]["timestamp"] ) # just checking 

ratings.head(5)

# we also load titles and create an id2title dictionnary
titleCSV = pd.read_csv("dataset/movies.csv")
titleCSV.head(5)
id2title = titleCSV[["movieId","title"]].set_index("movieId").to_dict()["title"]

True


In [13]:
ratings.head(5)

,userId,movieId,rating,timestamp
66719,429,595,5.0,828124615
66716,429,588,5.0,828124615
66717,429,590,5.0,828124615
66718,429,592,5.0,828124615
66712,429,432,3.0,828124615


## (a) Create sequence datasets:
For this task, we need sequences of items as data:

## (Todo): extract all movie sequences (in chronological order) from the dataset:


In this dataset, each user has seen at least 20 movies.


- We need to extract all movie rating sequences (there is one per user) from the dataset:

`sequence_of_movies = [[movieid,...],[movieid,...],...]`

In [14]:
sequences_of_movies = [i[1].tolist() for i in ratings.groupby(["userId"])["movieId"]]

## (Todo): Create a train/test dataset

Here, we propose as task to predict the last 5 items of each sequence.

In [15]:
train_seq,test_seq = [],[]

for seq in sequences_of_movies:
    train_seq.append(seq[:-5])
    test_seq.append(seq[-5:])
    
last_consumed_item = [seq[-1] for seq in train_seq] # We save the last consumed item for each list
                                                    # We'll use it as a starting point

## (Todo): Create the list of the most popular movies

- Here, popular is the number of times the movie appears in a list

In [16]:
from collections import Counter

counts = Counter([item for sublist in sequences_of_movies for item in sublist])
most_popular = np.array(counts.most_common(len(counts)))[:,0].tolist()
num_items = len(most_popular)

``` python
#Most popular looks like this:
[356,318,296,2571,593,260,480,110,589,...]
 ```

## Word2Vec skip-gram <=> Prod2Vec


### Word2Vec

The MAIN idea of word2vec is to maximise the similarity (dot product) between the vectors for words which appear close together (in the context of each other) in text, and minimise the similarity of words that do not. 

This can be applied to products instead of words: it clusters similar products together.


#### Paper Abstract:
> In recent years online advertising has become increasingly ubiquitous and effective. Advertisements shown to visitors fund sites and apps that publish digital content, manage social networks, and operate e-mail services. Given such large variety of internet resources, determining an appropriate type of advertising for a given platform has become critical to financial success. Native advertisements, namely ads that are similar in look and feel to content, have had great success in news and social feeds. However, to date there has not been a winning formula for ads in e-mail clients. In this paper we describe a system that leverages user purchase history determined from e-mail receipts to deliver highly personalized product ads to Yahoo Mail users. We propose to use a novel neural language-based algorithm specifically tailored for delivering effective product recommendations, which was evaluated against baselines that included showing popular products and products predicted based on co-occurrence. We conducted rigorous offline testing using a large-scale product purchase data set, covering purchases of more than 29 million users from 172 e-commerce websites. Ads in the form of product recommendations were successfully tested on online traffic, where we observed a steady 9% lift in click-through rates over other ad formats in mail, as well as comparable lift in conversion rates. Following successful tests, the system was launched into production during the holiday season of 2014

[Prod2Vec Model](https://arxiv.org/abs/1606.07154)



## Gensim has the best python implementation of word2vec's algorithms:

We can just use these raw implementations. The only thing to do is to consider items as words:

In [17]:
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)



train_seq_str = [list(map(str,seq)) for seq in train_seq] # we just say that our items id's are strings..
    

# the following configuration is the default configuration
w2v = gensim.models.word2vec.Word2Vec(sentences=train_seq_str,
                                size=50, window=10,               ### here we train a cbow model 
                                min_count=0,                      
                                sample=0.001, ns_exponent=0.75, workers=10,
                                sg=1, hs=0, negative=15,          ### set sg to 1 to train a sg model => Prod2Vec
                                cbow_mean=0,
                                iter=50)

2021-03-10 14:06:49,269 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2021-03-10 14:06:49,271 : INFO : collecting all words and their counts
2021-03-10 14:06:49,273 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-03-10 14:06:49,291 : INFO : collected 9616 word types from a corpus of 97786 raw words and 610 sentences
2021-03-10 14:06:49,293 : INFO : Loading a fresh vocabulary
2021-03-10 14:06:49,346 : INFO : effective_min_count=0 retains 9616 unique words (100% of original 9616, drops 0)
2021-03-10 14:06:49,347 : INFO : effective_min_count=0 leaves 97786 word corpus (100% of original 97786, drops 0)
2021-03-10 14:06:49,382 : INFO : deleting the raw counts dictionary of 9616 items
2021-03-10 14:06:49,383 : INFO : sample=0.001 downsamples 5 most-common words
2021-03-10 14:06:49,385 : INFO : downsampling leaves estimated 97657 word corpus (99.9% of prior 97786)
2021-03-10 14:06:49,410 : INFO : estimated required memory fo

2021-03-10 14:06:58,450 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-10 14:06:58,453 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-10 14:06:58,454 : INFO : EPOCH - 7 : training on 97786 raw words (97643 effective words) took 0.9s, 105675 effective words/s
2021-03-10 14:06:59,568 : INFO : EPOCH 8 - PROGRESS: at 13.28% examples, 9018 words/s, in_qsize 9, out_qsize 1
2021-03-10 14:06:59,569 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-10 14:06:59,600 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-10 14:06:59,608 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-10 14:06:59,610 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-10 14:06:59,620 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-10 14:06:59,621 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-10 14:06:59,622 : IN

2021-03-10 14:07:07,157 : INFO : EPOCH 15 - PROGRESS: at 13.28% examples, 9206 words/s, in_qsize 9, out_qsize 1
2021-03-10 14:07:07,158 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-10 14:07:07,171 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-10 14:07:07,179 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-10 14:07:07,187 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-10 14:07:07,213 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-10 14:07:07,218 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-10 14:07:07,220 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-03-10 14:07:07,222 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-10 14:07:07,227 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-10 14:07:07,232 : INFO : worker thread finished; awaiting fin

2021-03-10 14:07:14,644 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-10 14:07:14,650 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-10 14:07:14,656 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-10 14:07:14,662 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-10 14:07:14,669 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-03-10 14:07:14,673 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-10 14:07:14,675 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-10 14:07:14,680 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-10 14:07:14,681 : INFO : EPOCH - 22 : training on 97786 raw words (97669 effective words) took 1.0s, 100025 effective words/s
2021-03-10 14:07:15,617 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-10 14:07:15,629 : INFO : worker thread 

2021-03-10 14:07:22,026 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-10 14:07:22,029 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-10 14:07:22,041 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-10 14:07:22,048 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-10 14:07:22,049 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-10 14:07:22,058 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-03-10 14:07:22,065 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-10 14:07:22,066 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-10 14:07:22,072 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-10 14:07:22,073 : INFO : EPOCH - 30 : training on 97786 raw words (97666 effective words) took 0.9s, 103058 effective words/s
2021-03-10 14:07:23,000 : INFO : worker thread 

2021-03-10 14:07:29,103 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-10 14:07:29,115 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-10 14:07:29,121 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-10 14:07:29,122 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-10 14:07:29,126 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-10 14:07:29,129 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-10 14:07:29,133 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-03-10 14:07:29,135 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-10 14:07:29,136 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-10 14:07:29,139 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-10 14:07:29,139 : INFO : EPOCH - 38 : training on 97786 raw words (97650 effective

2021-03-10 14:07:35,760 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-10 14:07:35,761 : INFO : EPOCH - 45 : training on 97786 raw words (97667 effective words) took 0.9s, 110667 effective words/s
2021-03-10 14:07:36,837 : INFO : EPOCH 46 - PROGRESS: at 2.46% examples, 8418 words/s, in_qsize 9, out_qsize 1
2021-03-10 14:07:36,837 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-10 14:07:36,857 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-10 14:07:36,869 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-10 14:07:36,880 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-10 14:07:36,899 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-10 14:07:36,900 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-10 14:07:36,906 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-03-10 14:07:36,916 : I

### A few things:

In [18]:
print(w2v.wv.vectors[0])              # The vector of index 0
print(w2v.wv.index2word[0])           # codes for the movieId 356
print(w2v.wv.vocab["356"].index)      # Inverse mapping


[ 3.94260615e-01  1.59086823e-01  4.36876923e-01 -1.97362229e-01
 -2.18700424e-01 -6.10750794e-01 -6.15335166e-01 -4.72868860e-01
  6.49634421e-01 -4.43015881e-02  1.03993714e-01 -1.45449378e-02
 -3.98581266e-01  4.17220563e-01  8.00067931e-02 -1.52845219e-01
  3.23591352e-01  3.50121945e-01 -5.36270477e-02  5.83886266e-01
 -2.98651159e-01  1.49118587e-01 -1.69977576e-01 -2.59271294e-01
 -2.45543924e-06  2.60050684e-01  3.96595970e-02  1.25590324e-01
  1.20305479e-01 -6.05447851e-02 -3.50297123e-01 -2.53024578e-01
 -2.90223844e-02 -2.74757445e-01  6.47268474e-01  4.44178171e-02
  3.42183322e-01  8.37778628e-01  7.14502335e-02  1.31059498e-01
 -5.34952916e-02  4.93892908e-01  6.92009628e-01  7.61877239e-01
  2.35814303e-01 -5.91364913e-02 -6.41248941e-01  2.91897476e-01
  1.06868470e+00 -1.95978731e-01]
356
0


## Getting similar items:

The heart of the algorithm is in the similar item search. As in word2vec, we simply use cosine distance between items to find "similar items"

### We can search by id's

In [19]:
def get_similar_ids(w2vmodel,iid,num=5):
    
    if str(iid) in w2vmodel.wv.vocab:
        return [int(iid) for iid,_ in w2vmodel.wv.most_similar(str(iid),topn=num)] 
    else:
        return []

get_similar_ids(w2v,last_consumed_item[0],num=5)

2021-03-10 14:07:40,675 : INFO : precomputing L2-norms of word weight vectors


[26142, 66915, 27829, 26195, 44241]

### Or by vector

In [20]:
w2v.wv[str(last_consumed_item[0])]

array([ 0.49709412,  0.12318099,  0.2547634 , -0.2854433 ,  0.8169838 ,
       -0.00593364, -0.06113341,  0.41262147,  0.01466568, -0.3207735 ,
       -1.8353463 ,  0.31282464, -0.570262  ,  0.6451977 , -0.65667695,
       -0.4109201 ,  0.2599896 ,  1.1162055 , -1.0568888 ,  0.6207373 ,
       -0.549457  , -0.48206872, -0.3003626 , -0.7505662 ,  0.5699458 ,
       -0.43911052, -0.20870365,  0.05336251, -0.27725694,  0.19240242,
       -1.6434628 ,  1.353952  ,  0.5834888 ,  0.43346962, -0.13974561,
       -0.66950804,  1.3226165 ,  1.4049426 ,  0.42214537,  0.52268046,
        0.37544382,  1.1386514 ,  0.9060168 , -0.09290173,  0.3519177 ,
        0.00902228,  0.01525932,  0.17131856,  0.6134191 , -0.83163404],
      dtype=float32)

In [21]:
def get_similar_vectors(w2vmodel,vec,num=5):
        return [int(iid) for iid,_ in w2vmodel.wv.most_similar(positive=[vec],topn=num)] 

get_similar_vectors(w2v,w2v.wv[str(last_consumed_item[0])],num=5) # items are strings

[157, 26142, 66915, 27829, 26195]

### Let's see if this works

We can query by id

In [22]:
ID = 1
NUM_SIM = 3

print("Movies similar to: ", id2title[ID])
print("")
for x in get_similar_ids(w2v,ID,NUM_SIM):
    print("--> ",id2title[x])

Movies similar to:  Toy Story (1995)

-->  Toy Story 2 (1999)
-->  Forrest Gump (1994)
-->  Aladdin (1992)


We can also query by vector

**NOTE:** the 1st results can be the item(s) you've used to query

In [23]:
ID = 1
NUM_SIM = 4
print("Movies similar to: ", id2title[ID])
print("")
for x in get_similar_vectors(w2v,w2v.wv[str(ID)],NUM_SIM):
    print("--> ",id2title[x])

Movies similar to:  Toy Story (1995)

-->  Toy Story (1995)
-->  Toy Story 2 (1999)
-->  Forrest Gump (1994)
-->  Aladdin (1992)


Our result was the following (for ID = 1 & NUM_SIM = 3)

>Movies similar to:  Toy Story (1995)
>-  Beauty and the Beast (1991)
-   Toy Story 2 (1999)
-   Lion King, The (1994)


Using vectors enables operations like additions to be made

In [24]:
ID1 = 2571
ID2 = 589
NUM_SIM = 10

vec = np.max([w2v.wv[str(ID1)],w2v.wv[str(ID2)]],axis=0)# + w2v.wv[str(318)]

print("Movies similar to: ", id2title[ID1] , "+",  id2title[ID2] )
print("")
for x in get_similar_vectors(w2v,vec ,NUM_SIM):
    print("--> ",id2title[x])

Movies similar to:  Matrix, The (1999) + Terminator 2: Judgment Day (1991)

-->  Matrix, The (1999)
-->  Saving Private Ryan (1998)
-->  Terminator, The (1984)
-->  Terminator 2: Judgment Day (1991)
-->  Braveheart (1995)
-->  Sixth Sense, The (1999)
-->  Silence of the Lambs, The (1991)
-->  Star Wars: Episode V - The Empire Strikes Back (1980)
-->  Die Hard (1988)
-->  Forrest Gump (1994)


Ok, we now have a good base for our sequence recommendation algorithm, let's write something to evaluate our predictions

## (Todo) write a `get_relevance_list(proposed_ids,real_ids)` function:

This function will be used to compare proposed items w/ real items:


- A relevant item is an item which is in the ground truth
- It returns a list which length is the number of proposed items filled of 0's and 1's : 0 means the item is not relevant, 1 means it's relevant.

- get_relevance_list([1,2,3,4],[1,4,5,6]) should returns [1,0,0,1]  because items 1 and 4 are relevant.


In [25]:
def get_relevance_list(proposed_ids,real_ids):
    real_ids = set(real_ids)
    return [1 if i in real_ids else 0 for i in proposed_ids]
get_relevance_list([1,2,3,4],[1,4,5,6]) #returns [1,0,0,1]

[1, 0, 0, 1]

### Let's test our function on our data

In [26]:
get_relevance_list(most_popular[:25],test_seq[1])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [27]:
get_relevance_list(get_similar_ids(w2v,last_consumed_item[0],25),test_seq[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

## Ok, now, let's write prediction funtions:

- `predict_pop` will recommend the k's most popular items
- `predict_w2v` will recommend the k's most similar items to the last one consumed

#### (TODO) : complete those functions

In [28]:
def predict_pop(last_seen,k):
    return most_popular[:k]

def predict_w2v(last_seen,k):
    return get_similar_vectors(w2v,w2v.wv[str(last_seen)],num=k)

#data is list of last_consumed:
def get_predictions(predict_func,data,truth,k=5):
    if k == -1 or k == 0:
        k = num_items
    return [get_relevance_list(predict_func(last_seen,k),will_see) for last_seen,will_see in zip(data,truth)]

**Note**: The `get_predictions(...)` function returns the relevant list associated to predictions

### The following cells should return list of lists

In [29]:
print(get_predictions(predict_pop,last_consumed_item[:5],test_seq[:5],3))
print(get_predictions(predict_w2v,last_consumed_item[:5],test_seq[:5],3))

[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]


expected output: 
```
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]

```

## The return of the MRR and nDCG functions

In [30]:
test_list = [[0,0,1],[0,1,0],[1,0,0],[0,0,0]]

def rr(list_items):
    relevant_indexes = np.asarray(list_items).nonzero()[0]
    
    if len(relevant_indexes) > 0:
        return 1/(relevant_indexes[0]+1) # arrays are indexed from 0
    else:
        return 0

def mrr(list_list_items):
    return np.mean([rr(list_item) for list_item in list_list_items])

print(mrr(test_list)) #0.4583333333333333

# The dcg@k is the sum of the relevance, penalized gradually
def dcg_at_k(r, k):
    """Score is discounted cumulative gain (dcg)
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        
    """
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        
    return 0.

# test values
# r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
# dcg_at_k(r, 1) => 3.0
# dcg_at_k(r, 2) => 4.2618595071429155
r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
print(dcg_at_k(r, 1))
print(dcg_at_k(r, 2))

def mean_dcg(rel_lists,k):
    return np.mean([dcg_at_k(rel_list,k) for rel_list in rel_lists])

# And it's normalized version
def ndcg_at_k(r, k):
    """
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
    """
    dcg_max =  dcg_at_k(sorted(r)[::-1],k) 
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

# test values
# r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
# ndcg_at_k(r, 1) => 1.0
# ndcg_at_k(r, 4) => 0.794285
    
r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]    
print(ndcg_at_k(r, 4))

def mean_ndcg(rel_lists,k):
    return np.mean([ndcg_at_k(rel_list,k) for rel_list in rel_lists])

0.4583333333333333
3.0
4.2618595071429155
0.7942854176010882


## Let's see how this naïve way of predicting items to show works

In [31]:
pop_preds = get_predictions(predict_pop,last_consumed_item,test_seq,-1)
w2v_preds = get_predictions(predict_w2v,last_consumed_item,test_seq,-1)

print("1/MRR")
print(1/mrr(pop_preds))
print(1/mrr(w2v_preds))
print("")
print("DCG")
print(mean_dcg(pop_preds,5))
print(mean_dcg(w2v_preds,5))
print("")
print("nDCG")
print(mean_ndcg(pop_preds,5))
print(mean_ndcg(w2v_preds,5))

1/MRR
17.869504596139933
18.050316961446104

DCG
0.051054704276776476
0.055976629563232135

nDCG
0.017315723982695274
0.01924729183808887


### (TODO) Can we do better ?

Now, try a different strategy: 


- History should be discarded from prediction
- Instead of basing the prediction on the last seen item, we'll take all the `seen[-n:]` ones (horizon) into account
- To aggregate all items, we'll simply take the min rank to take into account the history offset.
- Equal scores can be handled using the history offset.
Example: 

> Let's say you chose to use the two last seen items `[item 44, Item 398]` to predict the following items

Therefore, using `get_similar_ids` method on both items will yield two lists of **similar** ranked item id's:
 - Similar to item 44: `[item 1, item 33, item 5]`
 - Similar to item 398: `[item 25, item 1, item 5]`
 scores (rank,offset):
 ```
 scores := {item 1: (0,0), item 33: (1,0), item 5: (2,0) , item 25: (0,1)}```
 
 
Then, aggregation by best rank should yield: `[1,25,33,5]`


In [84]:
print(get_similar_ids(w2v,44,num=5))
print(get_similar_ids(w2v,398,num=5))

[172, 204, 168, 511, 504]
[]


In [89]:
a = [9,8,7,6,5]
b = [8,7,6,5,4]
similar_items = [a,b]

items_ids = np.unique(liste)

scores = { item : ([],[]) for item in items_ids }
    
for i in range(len(similar_items)):
    for j in range(len(similar_items[i])):
        scores[similar_items[i][j]][0].append(j)
        scores[similar_items[i][j]][1].append(i)
                
        
scores = {k:(v[0],v[1]) for k,v in scores.items()}
            
sorted_by_second = sorted(scores.items(), key=lambda tup: tup[1])
scores

{4: ([4], [1]),
 5: ([4, 3], [0, 1]),
 6: ([3, 2], [0, 1]),
 7: ([2, 1], [0, 1]),
 8: ([1, 0], [0, 1]),
 9: ([0], [0])}

In [92]:
import time

def predict_max_w2v(seen,k,horizon=2):
    last_seen = seen[-horizon:]
    similars = [get_similar_ids(w2v,l,num=k) for l in last_seen]
    valeurs = np.unique(similars)
    scores = { v : ([],[]) for v in valeurs }
    
    for i in range(len(liste)):
        for j in range(len(liste[i])):
            #si c'est les listes sont vides 
            if not scores[liste[i][j]][0]:       
                scores[liste[i][j]][0].append(j)
                scores[liste[i][j]][1].append(i)

            elif scores[liste[i][j]][0][0] > j:
                scores[liste[i][j]][0][0] = j
                scores[liste[i][j]][1][0] = i

            elif scores[liste[i][j]][0][0] == j and scores[liste[i][j]][1][0] > i:
                scores[liste[i][j]][0][0] = j
                scores[liste[i][j]][1][0] = i
                
    sorted_by_second = sorted(scores.items(), key=lambda tup: tup[1])
    return [i[0] for i in sorted_by_second]

start_time = time.time()

#w2v_best_preds = get_predictions(predict_max_w2v,train_seq,test_seq,-1)
w2v_max_preds = get_predictions(predict_max_w2v,train_seq,test_seq,-1)

print(1/mrr(w2v_max_preds))
print(mean_dcg(w2v_max_preds,5))
print(mean_ndcg(w2v_max_preds,5))
print("--- %s seconds ---" % (time.time() - start_time))

58.23020538238749
0.01408107262782802
0.004775739482926849
--- 67.03965401649475 seconds ---


#### => Not really better

## Let's visualize learned embeddings

Just like in the 1st practical, we propose to visualize learnt items embeddings with the [Tensorflow projector](https://projector.tensorflow.org/).

In [94]:
# This function saves embeddings (a numpy array) and associated labels into tsv files.

def save_embeddings(embs,dict_label,path="saved_word_vectors"):
    """
    embs is Numpy.array(N,size)
    dict_label is {str(word)->int(idx)} or {int(idx)->str(word)}
    """
    def int_first(k,v):
        if type(k) == int:
            return (k,v)
        else:
            return (v,k)

    np.savetxt(f"{path}_vectors.tsv", embs, delimiter="\t")

    #labels 
    if dict_label:
        sorted_labs = np.array([lab for idx,lab in sorted([int_first(k,v) for k,v in dict_label.items()])])
        print(sorted_labs)
        with open(f"{path}_metadata.tsv","w") as metadata_file:
            for x in sorted_labs: #hack for space
                if len(x.strip()) == 0:
                    x = f"space-{len(x)}"
                    
                metadata_file.write(f"{x}\n")

In [95]:
vec2title = {i:id2title[int(mid)] for i,mid in enumerate(w2v.wv.index2word)}

In [96]:
save_embeddings(w2v.wv.vectors,vec2title)

['Forrest Gump (1994)' 'Shawshank Redemption, The (1994)'
 'Pulp Fiction (1994)' ... 'From Beyond (1986)' 'Signal, The (2007)'
 'Splinter (2008)']


## How to:

- Now, [open this link](https://projector.tensorflow.org/), and select "load".
- look for saved_word_vectors_vectors.tsv and saved_word_vectors_metadata.tsv. 

=> These are respectively, the items latent representations and their labels

## Hyperparameters matter when using Word2Vec for Item recommendation:


> Skip-gram with negative sampling, a popular variant of Word2vec originally designed and tuned to create word embeddings for Natural Language Processing, has been used to create item embeddings with successful applications in recommendation. While these fields do not share the same type of data, neither evaluate on the same tasks, recommendation applications tend to use the same already tuned hyperparameters values, even if optimal hyperparameters values are often known to be data and task dependent. We thus investigate the marginal importance of each hyperparameter in a recommendation setting through large hyperparameter grid searches on various datasets. Results reveal that optimizing neglected hyperparameters, namely negative sampling distribution, number of epochs, subsampling parameter and window-size, significantly improves performance on a recommendation task, and can increase it by an order of magnitude. Importantly, we find that optimal hyperparameters configurations for Natural Language Processing tasks and Recommendation tasks are noticeably different. 

[Hyperparameters matter](https://arxiv.org/abs/1804.04212)

#### It turns out that  hyperparameters are really important for this task: especially the sampling parameter.  Try and learn multiple models to see how the ns_exponent parameter modifies the results:


In [97]:
# the following configuration is the default configuration
w2v = gensim.models.word2vec.Word2Vec(sentences=train_seq_str,
                                size=50, window=3,               ### here we train a cbow model 
                                min_count=0,                      
                                sample=0.001, ns_exponent=-0.4, workers=10,
                                sg=1, hs=0, negative=15,          ### set sg to 1 to train a sg model => Prod2Vec
                                cbow_mean=0,
                                iter=25)



2021-03-10 19:54:05,625 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2021-03-10 19:54:05,627 : INFO : collecting all words and their counts
2021-03-10 19:54:05,628 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-03-10 19:54:05,640 : INFO : collected 9616 word types from a corpus of 97786 raw words and 610 sentences
2021-03-10 19:54:05,641 : INFO : Loading a fresh vocabulary
2021-03-10 19:54:05,657 : INFO : effective_min_count=0 retains 9616 unique words (100% of original 9616, drops 0)
2021-03-10 19:54:05,658 : INFO : effective_min_count=0 leaves 97786 word corpus (100% of original 97786, drops 0)
2021-03-10 19:54:05,692 : INFO : deleting the raw counts dictionary of 9616 items
2021-03-10 19:54:05,693 : INFO : sample=0.001 downsamples 5 most-common words
2021-03-10 19:54:05,694 : INFO : downsampling leaves estimated 97657 word corpus (99.9% of prior 97786)
2021-03-10 19:54:05,718 : INFO : estimated required memory fo

2021-03-10 19:54:10,343 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-10 19:54:10,344 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-10 19:54:10,345 : INFO : EPOCH - 7 : training on 97786 raw words (97651 effective words) took 0.3s, 279640 effective words/s
2021-03-10 19:54:10,638 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-10 19:54:10,640 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-10 19:54:10,643 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-10 19:54:10,660 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-10 19:54:10,667 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-10 19:54:10,676 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-10 19:54:10,687 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-03-10 19:54:10,689 : INFO : worker thread f

2021-03-10 19:54:13,180 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-10 19:54:13,186 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-10 19:54:13,187 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-10 19:54:13,187 : INFO : EPOCH - 15 : training on 97786 raw words (97655 effective words) took 0.3s, 289623 effective words/s
2021-03-10 19:54:13,472 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-10 19:54:13,473 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-10 19:54:13,493 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-10 19:54:13,496 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-10 19:54:13,506 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-10 19:54:13,516 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-10 19:54:13,518 : INFO : worker thread 

2021-03-10 19:54:16,151 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-03-10 19:54:16,164 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-10 19:54:16,169 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-10 19:54:16,171 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-10 19:54:16,172 : INFO : EPOCH - 23 : training on 97786 raw words (97639 effective words) took 0.4s, 276668 effective words/s
2021-03-10 19:54:16,464 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-10 19:54:16,476 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-10 19:54:16,481 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-10 19:54:16,499 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-10 19:54:16,510 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-10 19:54:16,511 : INFO : worker thread 

In [98]:
pop_preds = get_predictions(predict_pop,last_consumed_item,test_seq,-1)
w2v_preds = get_predictions(predict_w2v,last_consumed_item,test_seq,-1)

2021-03-10 19:54:20,193 : INFO : precomputing L2-norms of word weight vectors


In [99]:
print(1/mrr(pop_preds))
print(1/mrr(w2v_preds))

print(mean_dcg(pop_preds,5))
print(mean_dcg(w2v_preds,5))

print(mean_ndcg(pop_preds,5))
print(mean_ndcg(w2v_preds,5))

17.869504596139933
18.00438865481019
0.051054704276776476
0.06031670552534219
0.017315723982695274
0.020588149559852255


## Still got time ? Try making a more clever item selection mechanism:

- You could, for example, cluster items in groups (using k-means) and propose the most popular items of the last seen group